In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

img_size = 299
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while(True):
    ret, frame = cam.read()
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    for (x,y,w,h) in faces:
        if w >130:
            detected_face = frame[int(y):int(y+h), int(x):int(x+w)] 
            cv2.imshow("Test Photo", detected_face)
    if not ret:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:
        # ESC pressed
        print("Hit esc to skip and hit the spacebar to save image")
        break
    elif k%256 == 32:
        # SPACE pressed 
        faceresize = cv2.resize(detected_face, (img_size,img_size)) 
        img_name = "data/via/opencv_frame_{}.jpg".format(img_counter)
        cv2.imwrite(img_name, faceresize)
        print("{} written!".format(img_name))
        img_counter += 1
    
cam.release()

cv2.destroyAllWindows()

In [ ]:
train_path = 'data/trainimgface.csv'
test_path = 'data/testimgface.csv'
train_data = pd.read_csv(train_path)  
test_data = pd.read_csv(test_path)

In [ ]:
train_data.head()

In [ ]:
train_data.shape[1]

In [ ]:
test_data.head()

In [ ]:
coltrn = train_data['image']
imgs = []
print (coltrn.shape[0])
training = train_data.drop('image',axis = 1)
Y_train = []
for i in range (coltrn.shape[0]):
    p = os.path.join(os.getcwd(), 'data/'+str(coltrn.iloc[i]))
    img = cv2.imread(p, 1)
    print(img)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgs.append(gray_img)
    
    y = training.iloc[i,:]
    Y_train.append(y)

X_train = np.asarray(imgs)

Y_train = np.array(Y_train,dtype = 'float')  

print(X_train.shape, Y_train.shape)

In [ ]:
coltst = test_data['image']
imgstst = []
print (coltst.shape[0])
testing = test_data.drop('image',axis = 1)
Y_test = []
for i in range (coltst.shape[0]):
    p = os.path.join(os.getcwd(), 'data/'+str(coltst.iloc[i]))
    imgtst = cv2.imread(p, 1)
    gray_imgtst = cv2.cvtColor(imgtst, cv2.COLOR_BGR2GRAY)
    imgstst.append(gray_imgtst)
    
    y = testing.iloc[i,:]
    Y_test.append(y)

X_test = np.asarray(imgstst)

Y_test = np.array(Y_test,dtype = 'float')  

print(X_test.shape, Y_test.shape)

In [ ]:
Y_trainx = training.loc[:,['0x','1x','2x','3x','4x','5x','6x','7x','8x','9x','10x','11x','12x','13x','14x','15x']] 
Y_trainy = training.loc[:,['0y','1y','2y','3y','4y','5y','6y','7y','8y','9y','10y','11y','12y','13y','14y','15y']] 

print(Y_trainx.shape, Y_trainy.shape)
Y_trainx.head()

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(299,299,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))

model.add(Dense(32))

In [ ]:
model.summary()

In [ ]:
adam = Adam(lr=0.001)
model.compile(adam, loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
batchsize = 10
X_train= X_train.reshape(50,299,299,1)
X_test= X_test.reshape(7,299,299,1)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_test,Y_test), epochs=20, batch_size=batchsize)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_val = model.predict(X_test)

In [ ]:
yvalx = y_val[::1,::2]
yvaly = y_val[:, 1::2]